In [1]:
from knapsax.utils import setrootdir

setrootdir("knapsax")

'Directory knapsax successfully loaded as current working directory.'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from knapsax.optimization import Problem, Maximize, Solution, Knapsack
from knapsax.aco import ACO
from knapsax.stats import Stats

In [3]:
knapsack = Knapsack(instance_file="data/knapsack-instance.txt")
knapsack

Knapsack(file=data/knapsack-instance.txt, n_items=100, capacity=1550)

In [5]:
aco = ACO(
    knapsack=knapsack,
    n_ants=200, n_iterations=100, decay=0.95,
    pheromone_intensity=0.00001, n_best=3,
    alpha=1, beta=1
)

In [6]:
aco.run()

(array([2047, 2056, 2056, 2056, 2057, 2068, 2068, 2068, 2068, 2068, 2083,
        2083, 2083, 2090, 2090, 2090, 2090, 2091, 2091, 2091, 2105, 2115,
        2115, 2115, 2115, 2115, 2115, 2115, 2115, 2115, 2115, 2115, 2121,
        2121, 2129, 2129, 2129, 2137, 2137, 2137, 2137, 2137, 2137, 2137,
        2137, 2137, 2151, 2151, 2151, 2151, 2151, 2151, 2151, 2151, 2151,
        2156, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
        2164, 2164, 2181, 2181, 2181, 2181, 2181, 2181, 2181, 2181, 2181,
        2181, 2181, 2181, 2181, 2181, 2181, 2181, 2181, 2181, 2181, 2181,
        2181, 2181, 2181, 2181, 2186, 2186, 2186, 2186, 2186, 2187, 2187,
        2187], dtype=uint16),
 2187,
 1550)

In [7]:
aco_stats = Stats(
    n_runs=20,
    algorithm_instance=aco
)

aco_stats.run()

Running <class 'knapsax.aco.ACO'>:   0%|          | 0/20 [04:21<?, ?run/s]


KeyboardInterrupt: 

In [ ]:
df_aco = aco_stats.frame()
df_aco

In [ ]:
aco_stats.plot_convergence(
    title="Convergência do ACO para o problema da mochila",
    savefig="results/aco_convergence.png"
)